# An introduction to PCA

Using principal component analysis to reveal latent structure in data

PCA is often used to reduce the dimensionality of large data while
preserving a significant amount of variance. More fundamentally, it is a
framework for studying the covariance statistics of data. In this
section, we will introduce the concept of PCA with some toy examples.

## A simple experiment

Let’s perform an imaginary neuroscience experiment! We’ll record
voltages from $P = 2$ neurons in visual cortex while the participant
passively views $N = 1000$ dots of different *colors* and *sizes*.

In [ ]:
%pip install git+https://github.com/BonnerLab/ccn-tutorial.git

In [ ]:
from collections.abc import Sequence
import warnings
from typing import NamedTuple

import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib_inline.backend_inline import set_matplotlib_formats
import ipywidgets as widgets
from IPython.display import display, HTML


In [ ]:
%matplotlib inline

sns.set_theme(
    context="notebook",
    style="white",
    palette="deep",
)
set_matplotlib_formats("svg")

pd.set_option("display.max_rows", 5)
pd.set_option("display.max_columns", 10)
pd.set_option("display.precision", 3)
pd.set_option("display.show_dimensions", False)

xr.set_options(display_max_rows=3, display_expand_data=False)

warnings.filterwarnings("ignore")

In [ ]:
random_state = 0
rng = np.random.default_rng(seed=random_state)


### Creating the stimuli

Let’s create $N = 1000$ dots of different *colors* and *sizes*. From the
scatterplot, we can see that the two latent variables are
[uncorrelated](reference.qmd#correlation)).

In [ ]:
def create_stimuli(
    *,
    n_stimuli: int,
    rng: np.random.Generator,
) -> pd.DataFrame:
    return pd.DataFrame(
        {
            "color": rng.random(size=(n_stimuli,)),
            "size": rng.random(size=(n_stimuli,)),
        }
    ).set_index(1 + np.arange(n_stimuli))


def view_stimuli(data: pd.DataFrame) -> None:
    fig, ax = plt.subplots()
    sns.scatterplot(
        ax=ax,
        data=data,
        x="color",
        y="size",
        hue="color",
        size="size",
        palette="flare",
        legend=False,
    )
    sns.despine(ax=ax, trim=True)
    fig.tight_layout()
    fig.show()


stimuli = create_stimuli(n_stimuli=1_000, rng=rng)

view_stimuli(stimuli)


### Simulating neural responses

Now, let’s simulate some neural data. We need to decide how the $P = 2$
neurons might respond to these $N = 1000$ stimulus dots. Each neuron
could respond to either one or both of the latent features that define
these stimuli – $\text{color}$ and $\text{size}$. The neuron’s responses
could also be subject to noise. Hence, we model each neuron’s response
$r_\text{neuron}$ as a simple linear combination of the two latent
features with stimulus-independent Gaussian noise $\epsilon$:

$r_{\text{neuron}} \sim \beta_{\text{color}} \left( \text{color} \right) + \beta_{\text{size}} \left( \text{size} \right) + \epsilon$,
where
$\epsilon \sim \mathcal{N}(\mu_{\text{neuron}}, \sigma_{\text{neuron}}^2)$

This procedure produces a data matrix $X \in \mathbb{R}^{N \times P}$
containing the $P = 2$ neurons’ responses to the $N = 1000$ stimuli.

In [ ]:
Neuron = NamedTuple(
    "Neuron",
    beta_color=float,
    beta_size=float,
    mean=float,
    std=float,
)


def simulate_neuron_responses(
    stimuli: pd.DataFrame,
    neuron: Neuron,
    *,
    rng: np.random.Generator,
) -> np.ndarray:
    def z_score(x: np.ndarray) -> np.ndarray:
        return (x - x.mean()) / x.std()

    return (
        neuron.beta_color * z_score(stimuli["color"])
        + neuron.beta_size * z_score(stimuli["size"])
        + neuron.std * rng.standard_normal(size=(len(stimuli),))
        + neuron.mean
    )


def simulate_multiple_neuron_responses(
    *,
    stimuli: pd.DataFrame,
    neurons: Sequence[Neuron],
    rng: np.random.Generator,
) -> xr.DataArray:
    data = []
    for i_neuron, neuron in enumerate(neurons):
        data.append(
            xr.DataArray(
                data=simulate_neuron_responses(
                    stimuli=stimuli,
                    neuron=neuron,
                    rng=rng,
                ),
                dims=("stimulus",),
                coords={
                    column: ("stimulus", values)
                    for column, values in stimuli.reset_index(names="stimulus").items()
                },
            )
            .expand_dims({"neuron": [i_neuron + 1]})
            .assign_coords(
                {
                    field: ("neuron", [float(value)])
                    for field, value in neuron._asdict().items()
                }
            )
        )

    data = (
        xr.concat(data, dim="neuron")
        .rename("neuron responses")
        .transpose("stimulus", "neuron")
    )

    return data


neurons = (
    Neuron(beta_color=3, beta_size=-2, std=1, mean=7),
    Neuron(beta_color=-2, beta_size=5, std=3, mean=-6),
)

data = simulate_multiple_neuron_responses(
    stimuli=stimuli,
    neurons=neurons,
    rng=rng,
)

display(data)


### Visualizing the neurons

We can visualize the responses of each neuron to each dot. Note that
this is a 1-dimensional scatterplot; the spread along the vertical axis
is just for visualization purposes.

In [ ]:
def view_individual_responses(data: xr.DataArray, *, coord: str) -> None:
    rng = np.random.default_rng()
    data_ = data.assign_coords(
        {"arbitrary": ("stimulus", rng.random(data.sizes["stimulus"]))}
    )
    min_, max_ = data_.min(), data_.max()

    match coord:
        case "neuron":
            dim = "neuron"
            prefix = "neuron"
            suffix = "response"
        case "rank":
            dim = "component"
            prefix = "principal component"
            suffix = "score"
        case _:
            raise ValueError("coord must be 'neuron' or 'rank'")

    dim = data[coord].dims[0]
    n_features = data.sizes[dim]

    fig, axes = plt.subplots(nrows=n_features, figsize=(7, 2 * n_features))

    for value, ax in zip(data[coord].values, axes.flat):
        sns.scatterplot(
            ax=ax,
            data=(
                data_.isel({dim: data[coord].values == value})
                .rename(f"{prefix} {value} {suffix} (a.u.)")
                .to_dataframe()
            ),
            x=f"{prefix} {value} {suffix} (a.u.)",
            y="arbitrary",
            hue="color",
            size="size",
            palette="flare",
            legend=False,
        )
        sns.despine(ax=ax, left=True, offset=10)

        ax.set_xlim([min_, max_])
        ax.get_yaxis().set_visible(False)

    fig.tight_layout(h_pad=3)
    fig.show()


view_individual_responses(data, coord="neuron")


We can see that each neuron is tuned to both color *and* size.
Additionally, note that the neurons’ responses have different variances.

In [ ]:
variances = data.var(dim="stimulus", ddof=1).round(3).rename("neuron variances")
for i_neuron in range(variances.sizes["neuron"]):
    print(
        f"variance of neuron {i_neuron + 1} responses:"
        f" {variances[i_neuron].values} (a.u.)^2"
    )
print(f"total variance: {variances.sum().values} (a.u.)^2")


## Understanding the neural code

### Linear encoding models

### Representational similarity analysis

## Studying the latent dimensions

We have a population of neurons that contain information about the
stimulus: that is, from the activity pattern we recorded, we expect to
be able to reliably decode the color and size of the dot presented. How
is this information encoded in the population activity? Is there a
neuron that is sensitive to color and another that is sensitive to size?
Or are there neural responses more complicated than this? If so, is
there another view of the population code that might be more
informative?

### Some geometric intuition

Since we only have $P = 2$ neurons, we can visualize these data as a
scatterplot, which makes their [covariance](reference.qmd#covariance)
apparent.

In [ ]:
def view_joint_neuron_responses(data: xr.DataArray) -> None:
    fig, ax = plt.subplots()

    data_ = pd.DataFrame(
        {coord: data[coord].values for coord in ("color", "size")}
        | {
            f"neuron {neuron} response (a.u.)": data.sel(neuron=neuron).to_dataframe()[
                "neuron responses"
            ]
            for neuron in (1, 2)
        }
    )
    sns.scatterplot(
        ax=ax,
        data=data_,
        x="neuron 1 response (a.u.)",
        y="neuron 2 response (a.u.)",
        hue="color",
        size="size",
        legend=False,
        palette="flare",
    )
    ax.axhline(0, c="gray", ls="--")
    ax.axvline(0, c="gray", ls="--")

    ax.axis("square")
    sns.despine(ax=ax, offset=20)

    return fig


fig = view_joint_neuron_responses(data)


The organization of these data in this 2-dimensional space suggests an
obvious way to change our viewpoint.

In [ ]:
def view_pca_animation(
    data: xr.DataArray,
    *,
    durations: dict[str, int] = {
        "center": 1_000,
        "rotate": 1_000,
        "pause": 1_500,
    },
    interval: int = 50,
) -> None:
    def _compute_2d_rotation_matrix(theta: float) -> np.ndarray:
        return np.array(
            [
                [np.cos(theta), -np.sin(theta)],
                [np.sin(theta), np.cos(theta)],
            ]
        )

    fig = view_joint_neuron_responses(data)
    ax = fig.get_axes()[0]
    scatter = ax.get_children()[0]
    title = fig.suptitle("neuron responses")

    n_frames = {key: value // interval + 1 for key, value in durations.items()}

    x_mean, y_mean = data.mean("stimulus").values
    delta = np.array([x_mean, y_mean]) / n_frames["center"]

    _, _, v_h = np.linalg.svd(data - data.mean("stimulus"))
    v = v_h.transpose()
    theta = np.arccos(v[0, 0])
    rotation = _compute_2d_rotation_matrix(-theta / n_frames["rotate"])

    transformed = (data - data.mean("stimulus")).values @ v

    radius = max(np.linalg.norm(transformed, axis=-1))
    limit = max(np.abs(data).max(), np.abs(transformed).max(), radius)
    ax.set_xlim([-limit, limit])
    ax.set_ylim([-limit, limit])
    fig.tight_layout()

    frame_to_retitle_center = 2 * n_frames["pause"]
    frame_to_start_centering = frame_to_retitle_center + n_frames["pause"]
    frame_to_stop_centering = frame_to_start_centering + n_frames["center"]
    frame_to_retitle_rotate = frame_to_stop_centering + n_frames["pause"]
    frame_to_start_rotating = frame_to_retitle_rotate + n_frames["pause"]
    frame_to_stop_rotating = frame_to_start_rotating + n_frames["rotate"]
    frame_to_retitle_transformed = frame_to_stop_rotating + n_frames["pause"]
    frame_to_end = frame_to_retitle_transformed + 2 * n_frames["pause"]

    def _update(frame: int) -> None:
        if frame < frame_to_retitle_center:
            return
        elif frame == frame_to_retitle_center:
            title.set_text("center the data")
            ax.set_xlabel("")
            ax.set_ylabel("")
        elif frame < frame_to_start_centering:
            return
        elif frame <= frame_to_stop_centering:
            scatter.set_offsets(scatter.get_offsets() - delta)
        elif frame == frame_to_retitle_rotate:
            title.set_text("rotate the data")
        elif frame < frame_to_start_rotating:
            return
        elif frame <= frame_to_stop_rotating:
            scatter.set_offsets(scatter.get_offsets().data @ rotation)
        elif frame < frame_to_retitle_transformed:
            return
        elif frame == frame_to_retitle_transformed:
            title.set_text("principal components")
            ax.set_xlabel("principal component 1")
            ax.set_ylabel("principal component 2")
        elif frame <= frame_to_end:
            return

    animation = FuncAnimation(
        fig=fig,
        func=_update,
        frames=frame_to_end,
        interval=interval,
        repeat=True,
        repeat_delay=2_500,
    )
    display(HTML(animation.to_html5_video()))
    plt.close(fig)


view_pca_animation(data)


### The mathematical definition

Given a data matrix $X \in \mathbb{R}^{N \times P}$, we need to compute
the [eigendecomposition](reference.qmd#eigendecomposition)[1] of its
[auto-covariance](reference.qmd#auto-covariance)[2]:

$$
\begin{align}
    \text{cov}(X)
    &= \left(\dfrac{1}{n - 1}\right) (X - \overline{X})^\top (X - \overline{X})\\
    &= V \Lambda V^\top
\end{align}
$$

The columns of $V$ are *eigenvectors* that specify the directions of
variance while the corresponding diagonal elements of $\Lambda$ are
*eigenvalues* that specify the amount of variance along the
eigenvector[3].

The original data matrix can be transformed by projecting it onto the
eigenvectors: $\tilde{X} = \left(X - \overline{X}\right) V$.

> **Viewing PCA as an optimization**
>
> PCA can be used to project data into a lower-dimensional space
> (i.e. $p \le f$) in a way that best preserves the geometry of the
> data. Specifically, computing a PCA decomposition of $X$ yields a
> matrix $V \in \mathbb{R}^{f \times p}$ such that
> $V = \argmin_{V \in \mathbb{U_{f \times p}}} \sum_{i=1}^n \left|| x_i - VV^\top x_i \right||_2$,
> where $||\cdot||_2$ denotes the $L_2$-norm and
> $\mathbb{U_{f \times p}}$ denotes the set of orthonormal matrices with
> shape $f \times p$.

### Transforming the data

[1] The eigendecomposition of a symmetric matrix
$X \in \mathbb{R}^{n \times n}$ involves rewriting it as the product of
three matrices $X = V \Lambda V^\top$, where $V \in \mathbb{n \times n}$
is orthonormal and $\Lambda \in \mathbb{n \times n}$ is diagonal with
non-negative entries.

[2] Given a data matrix $X \in \mathbb{R}^{n \times f}$ containing
neural responses to $n$ stimuli from $f$ neurons, the *auto-covariance*
of $X$ (or simply its *covariance*) is defined as:

$\text{cov}(X) = \left(\dfrac{1}{n - 1}\right) (X - \overline{X})^\top (X - \overline{X})$

This is an $f \times f$ matrix where the $(i, j)$-th element measures
how much neuron $i$ covaries with neuron $j$. If the covariance is
positive, they tend to have similar activation: a stimulus that
activates one neuron will tend to activate the other. If the covariance
is negative, the neurons will have dissimilar activation: a stimulus
that activates one neuron will likely not activate the other.

[3] Let’s compute the auto-covariance of the projected data $\tilde{X}$:

$$
\begin{align}
    \text{cov}(\tilde{X})
    &= \left(\dfrac{1}{n - 1}\right) \tilde{X}^\top \tilde{X}\\
    &= \left(\dfrac{1}{n - 1}\right) \left((X - \overline{X})V\right)^\top \left((X - \overline{X})V\right)\\
    &= \left(\dfrac{1}{n - 1}\right) V^\top (X - \overline{X})^\top (X - \overline{X})V\\
    &= V^\top \left(\dfrac{1}{n - 1}\right) (X - \overline{X})^\top (X - \overline{X})V\\
    &= V^\top \left( V \Lambda V^\top \right) V\\
    &= I \Lambda I\\
    &= \Lambda
\end{align}
$$

In [ ]:
class PCA:
    def __init__(self) -> None:
        self.mean: np.ndarray
        self.eigenvectors: np.ndarray
        self.eigenvalues: np.ndarray

    def fit(self, /, data: np.ndarray) -> None:
        self.mean = data.mean(axis=-2)

        data_centered = data - self.mean
        _, s, v_t = np.linalg.svd(data_centered)

        n_stimuli = data.shape[-2]

        self.eigenvectors = np.swapaxes(v_t, -1, -2)
        self.eigenvalues = s**2 / (n_stimuli - 1)

    def transform(self, /, data: np.ndarray) -> np.ndarray:
        return (data - self.mean) @ self.eigenvectors


> **Why do we compute PCA this way instead of
> $\text{eig}(\text{cov}(X))$?**
>
> To apply PCA to a data matrix, we might be tempted to use the
> definition and naively compute its
> [auto-covariance](reference.qmd#auto-covariance) followed by an
> [eigendecomposition](reference.qmd#eigendecomposition). However, when
> the number of neurons $P$ is large, this approach is memory-intensive
> and prone to numerical errors.
>
> Instead, we can use the [singular value
> decomposition](reference.qmd#singular-value-decomposition) (SVD) of
> $X$ to efficiently compute its PCA transformation. Specifically,
> $X = U \Sigma V^\top$ is a singular value decomposition, where $U$ and
> $V$ are orthonormal and $\Sigma$ is diagonal.
>
> The auto-covariance matrix reduces to
> $X^\top X / (n - 1) = V \left(\frac{\Sigma^2}{n - 1} \right) V^\top$,
> which is exactly the eigendecomposition required.
>
> Specifically, the eigenvalues $\lambda_i$ of the auto-covariance
> matrix are related to the singular values $\sigma_i$ of the data
> matrix as $\lambda_i = \sigma_i^2 / (N - 1)$, while the eigenvectors
> of the auto-covariance matrix are exactly the right singular vectors
> $V$ of the data matrix $X$.

In [ ]:
def compute_pca(data: xr.DataArray) -> xr.Dataset:
    pca = PCA()
    pca.fit(data.values)

    data_transformed = pca.transform(data.values)

    return xr.Dataset(
        data_vars={
            "score": xr.DataArray(
                data=data_transformed,
                dims=("stimulus", "component"),
            ),
            "eigenvector": xr.DataArray(
                data=pca.eigenvectors,
                dims=("component", "neuron"),
            ),
        },
        coords={
            "rank": ("component", 1 + np.arange(data_transformed.shape[-1])),
            "eigenvalue": ("component", pca.eigenvalues),
        }
        | {coord: (data[coord].dims[0], data[coord].values) for coord in data.coords},
    )


def view_transformed_data(pca: xr.DataArray) -> None:
    data_ = pd.DataFrame(
        {coord: pca[coord].values for coord in ("color", "size")}
        | {
            f"principal component {rank}": (
                pca["score"].sel(component=rank - 1).to_dataframe()["score"]
            )
            for rank in (1, 2)
        }
    )
    fig, ax = plt.subplots()
    sns.scatterplot(
        ax=ax,
        data=data_,
        x="principal component 1",
        y="principal component 2",
        hue="color",
        size="size",
        palette="flare",
        legend=False,
    )
    ax.axis("square")
    ax.axhline(0, c="gray", ls="--")
    ax.axvline(0, c="gray", ls="--")
    sns.despine(ax=ax, offset=20)

    fig.show()


data_transformed = compute_pca(data)
view_transformed_data(data_transformed)


Let’s view the data projected onto each of the principal components.

In [ ]:
view_individual_responses(data_transformed["score"], coord="rank")


We can observe that:

-   the *first* principal component is largely driven by the *size* of
    the stimulus
-   the *second* principal component is largely driven by the *color* of
    the stimulus

However, note that these components do not directly correspond to either
of the latent variables; rather, each is a mixture of stimulus-dependent
signal and noise.

### Analyzing the covariance statistics

We can compute the variance along each eigenvector. The total variance
along all eigenvectors is the same as the total variance of the original
data.

In [ ]:
eigenvalues = data_transformed["eigenvalue"].round(3)
for i_neuron in range(eigenvalues.sizes["component"]):
    print(
        f"variance along eigenvector {i_neuron + 1} (eigenvalue {i_neuron + 1}):"
        f" {eigenvalues[i_neuron].values}"
    )
print(f"total variance: {eigenvalues.sum().values}")


In [ ]:
def view_eigenspectrum(pca: xr.DataArray) -> None:
    fig, ax = plt.subplots(figsize=(pca.sizes["component"], 5))
    sns.lineplot(
        ax=ax,
        data=pca["component"].to_dataframe(),
        x="rank",
        y="eigenvalue",
        marker="s",
    )
    ax.set_xticks(pca["rank"].values)
    ax.set_ylim(bottom=0)
    sns.despine(ax=ax, offset=20)

    fig.show()


view_eigenspectrum(data_transformed)


## More neurons!

Let’s record from more neurons (say $P = 10$)! Here, the *ambient
dimensionality* of the feature space is $P = 10$.

In [ ]:
def _simulate_random_neuron(rng: np.random.Generator) -> Neuron:
    return Neuron(
        beta_color=rng.integers(-10, 11),
        beta_size=rng.integers(-10, 11),
        std=rng.integers(-10, 11),
        mean=rng.integers(-10, 11),
    )


neurons = tuple([_simulate_random_neuron(rng) for _ in range(10)])

big_data = simulate_multiple_neuron_responses(
    stimuli=stimuli,
    neurons=neurons,
    rng=rng,
)

display(big_data)


In [ ]:
big_data_transformed = compute_pca(big_data)
view_individual_responses(big_data_transformed["score"], coord="rank")


In [ ]:
view_eigenspectrum(big_data_transformed)


However, we know that this data was generated from exactly *2* latent
variables – *color* and *size*. We refer to these as the *effective
dimensions*. In this example, the remaining dimensions, which correspond
to noise, have relatively low variance, and we can see a signficant drop
in the eigenspectrum after the second eigenvector.

### Quantifying dimensionality

Based on the spectrum, several approaches are used to quantify the
latent dimensionality of a dataset:

#### Rank of the covariance matrix

The *rank* of the covariance matrix – equal to the number of nonzero
eigenvalues – would be the latent dimensionality in the ideal setting
where the data has zero noise. In real data, the rank is typically equal
to the ambient dimensionality, since there is typically some variance
along every dimension.

#### Setting an arbitrary variance threshold

Though not typically used today, another approach is to set an arbitrary
threshold on the variance (historically recommended as $1$ for
normalized data); only dimensions with variance above that threshold are
considered useful.

#### Setting an arbitrary *cumulative* variance threshold

A very commonly used method is to set a threshold based on the
cumulative variance of the data: the number of dimensions required to
exceed, say $80\%$ of the variance, is taken as the latent
dimensionality.

#### Eyeballing the “knee” of the spectrum

When the number of latent dimensions is low, eigenspectra often have a
sharp discontinuity (the “knee”), where a small number of dimensions
have high-variance and the remainder have much have lower variance. The
latent dimensionality is then taken to be the number of dimensions above
the threshold, determined by eye.

#### Computing a summary statistic over the entire spectrum

A metric such as *effective dimensionality* summarizes the spectrum
using an entropy-like measure, taking into account variances along all
the dimensions:

$$\text{effective dimensionality}(\lambda_1, \dots \lambda_n) = \dfrac{\left( \sum_{i=1}^n \lambda_i \right)^2}{\sum_{i=1}^n \lambda_i^2}$$

However, keep in mind that this is a toy example with idealized data. As
we will see, when using standard PCA on real data it may be impossible
to identify a clear distinction between meaningful dimensions and noise.

> **Only need the first few PCs?**
>
> Check out [truncated
> SVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)!